# Research on Borrower Reliability
Client: Bank's credit department. The task is to determine whether marital status and the number of children of a client affect the likelihood of timely loan repayment. The input data from the bank includes statistics on the clients' creditworthiness.

The research results will be taken into account when developing the credit scoring model — a special system that assesses the potential borrower's ability to repay the loan to the bank.

## Step 1. Open the data file and examine the general information

In [290]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
print(df.shape)
print(df.info())
display(df.head())

(21525, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Conclusion**

It is evident that there are missing values in the columns days_employed and total_income.

Additionally, there are negative values in the days_employed column. Employment duration cannot be negative.

In the education column, there are values with the same meaning but different capitalization.

## Step 2. Data Preprocessing

### Handling Missing Values


In the days_employed column, there are negative values. Employment duration cannot be negative. Perhaps, there was an error during the data extraction.

Let's correct the values by taking the absolute value using the abs() method.

In [291]:
df['days_employed'] = df['days_employed'].abs()

Now let's see which rows have missing values. Perhaps there is a pattern among them. Let's filter the days_employed column and display only the rows with missing values.

In [292]:
df_days_employed = df[df['days_employed'].isna()]
display(df_days_employed.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Let's see how many missing values are in each employment category.

In [293]:
df_income_type = df_days_employed.groupby('income_type')['income_type'].count()
df_income_type

income_type
госслужащий         147
компаньон           508
пенсионер           413
предприниматель       1
сотрудник          1105
Name: income_type, dtype: int64

There is a suspicion that all cells in the total_income column are also empty. Let's check it.

In [294]:
display(df_days_employed['total_income'].unique())
print(df_days_employed.shape[0])

array([nan])

2174


All right.

Now let's find the mean value of employment duration and income for each employment type.



In [295]:
df_avg = df.groupby(by='income_type')['days_employed', 'total_income'].mean()
display(df_avg)

,days_employed,total_income
income_type,,
безработный,366413.652744,131339.751676
в декрете,3296.759962,53829.130729
госслужащий,3399.896902,170898.309923
компаньон,2111.524398,202417.461462
пенсионер,365003.491245,137127.465690
предприниматель,520.848083,499163.144947
сотрудник,2326.499216,161380.260488
студент,578.751554,98201.625314



Let's rename the columns to avoid confusion when we merge them with the main table in the future.

In [296]:
df_avg.columns = ['days_employed_avg', 'total_income_avg']
df_avg

,days_employed_avg,total_income_avg
income_type,,
безработный,366413.652744,131339.751676
в декрете,3296.759962,53829.130729
госслужащий,3399.896902,170898.309923
компаньон,2111.524398,202417.461462
пенсионер,365003.491245,137127.465690
предприниматель,520.848083,499163.144947
сотрудник,2326.499216,161380.260488
студент,578.751554,98201.625314



We merge the two tables using the merge() method.

In [297]:
df = df.merge(df_avg, on='income_type', how='left')
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_avg,total_income_avg
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,2326.499216,161380.260488
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,2326.499216,161380.260488
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,2326.499216,161380.260488
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,2326.499216,161380.260488
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,365003.491245,137127.465690



We replace the missing values with the mean values for each employment group.

In [298]:
df['days_employed'] = df['days_employed'].fillna(df['days_employed_avg'])
df['total_income'] = df['total_income'].fillna(df['total_income_avg'])

We remove the last two columns. They are no longer needed.

In [299]:
df = df.drop('days_employed_avg', 1)
df = df.drop('total_income_avg', 1)

In [300]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Conclusion**

All missing values in the `days_employed` and `total_income` columns have been replaced with the mean value.

Negative values in the employment duration column have been changed to positive.

The reason for the appearance of missing values in the `days_employed` and `total_income` columns may be that these individuals are not employed officially.

### Data Type Conversion


Let's change the data type from float to int64.

In [301]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
print(df.info())
display(df.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB
None


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


**Conclusion**

We have removed the float data type. Next, we will work with int64.


### Handling Duplicates

First, let's convert all values in the `education` column to lowercase. Since it is already evident that there are values in the `education` column that have the same meaning but different capitalization.

In [302]:
df['education'] = df['education'].str.lower()
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Let's remove the explicit duplicates.

In [303]:
df = df.drop_duplicates()

**Conclusion**

In the `education` column, there were initially data with different capitalization. By using the `str.lower()` method, I removed values that have the same meaning.

Then, I removed all explicit duplicates.

Possibly, duplicates appear among individuals who have more than one loan.

### Lemmatization

In [304]:
from pymystem3 import Mystem
m = Mystem()
from collections import Counter
print(Counter(df['purpose']))

from nltk.stem import SnowballStemmer
russian_stemmer = SnowballStemmer('russian')

Counter({'свадьба': 791, 'на проведение свадьбы': 768, 'сыграть свадьбу': 765, 'операции с недвижимостью': 675, 'покупка коммерческой недвижимости': 661, 'операции с жильем': 652, 'покупка жилья для сдачи': 651, 'операции с коммерческой недвижимостью': 650, 'покупка жилья': 646, 'жилье': 646, 'покупка жилья для семьи': 638, 'строительство собственной недвижимости': 635, 'недвижимость': 633, 'операции со своей недвижимостью': 627, 'строительство жилой недвижимости': 624, 'покупка недвижимости': 621, 'покупка своего жилья': 620, 'строительство недвижимости': 619, 'ремонт жилью': 607, 'покупка жилой недвижимости': 606, 'на покупку своего автомобиля': 505, 'заняться высшим образованием': 496, 'автомобиль': 494, 'сделка с подержанным автомобилем': 486, 'на покупку подержанного автомобиля': 478, 'автомобили': 478, 'свой автомобиль': 478, 'на покупку автомобиля': 471, 'приобретение автомобиля': 461, 'дополнительное образование': 460, 'сделка с автомобилем': 455, 'высшее образование': 452, 'об

**Conclusion**

It is evident that the main purpose of the loan is wedding, followed by loans for purchasing/repairing housing, car, and education.

### Data Categorization

Let's create separate tables to answer each question.
1. Table with marital status
2. Table with the number of children

In [305]:
df_family_status = df.groupby(by='family_status')['debt'].sum() / df.groupby(by='family_status')['debt'].count() * 100
df_children = df.groupby(by='children')['debt'].sum() / df.groupby(by='children')['debt'].count() * 100

Let's write a function that will divide the data in the `total_income` column into 4 groups.

In [306]:
def income_group_fc(total_income):
    if total_income < 100000:
        return '< 100000'
    elif total_income <= 500000:
        return '100000 - 500000'
    elif total_income < 1000000:
        return '500001 - 1000000'
    else:
        return '> 1000001'

Let's create a new table based on `total_income`. Then, using the `apply` method and the `income_group_fc` function, we will add a new column with the income group.

In [307]:
df_total_income = df.loc[:, ['debt', 'total_income']]
df_total_income['income_group'] = df_total_income['total_income'].apply(income_group_fc)
df_total_income = df_total_income.groupby(by='income_group')['debt'].sum() / df_total_income.groupby(by='income_group')['debt'].count() * 100

**Conclusion**

Three new tables have been added to answer the main questions of the project.

## Step 3. Answer the Questions

- Is there a correlation between having children and repaying the loan on time?

In [308]:
display(df_children)

children
-1      2.127660
 0      7.543822
 1      9.234609
 2      9.454191
 3      8.181818
 4      9.756098
 5      0.000000
 20    10.526316
Name: debt, dtype: float64

**Conclusion**

If a client does not have children, they are more likely to repay the loan on time compared to clients with children. Moreover, the number of children does not seem to play a significant role.

It is unclear what "-1 child" means. Perhaps something happened... However, since this case is rare, it will not affect the overall conclusions.

- Is there a correlation between marital status and repayment of the loan on time?

In [309]:
display(df_family_status)

family_status
Не женат / не замужем    9.750890
в разводе                7.112971
вдовец / вдова           6.569343
гражданский брак         9.347145
женат / замужем          7.545182
Name: debt, dtype: float64

**Conclusion**

People categorized as "not married" and "civil marriage" are more likely to have overdue loans.

People categorized as widows/widowers are the best at repaying loans.

- Is there a correlation between income level and repayment of the loan on time?

In [310]:
display(df_total_income)

income_group
100000 - 500000     8.187727
500001 - 1000000    6.091371
< 100000            7.931884
> 1000001           8.000000
Name: debt, dtype: float64

**Conclusion**

The risk group includes people with incomes ranging from 100k to 500k.

The most creditworthy group turned out to be the group with incomes ranging from 500k to 1,000k.

- How do different loan purposes affect timely repayment?

Let's write a function to categorize loan purposes.

In [311]:
def df_purpose_fc(x):
    if 'авто' in x:
        return 'авто'
    elif 'недвиж' in x:
        return 'недвижимость'
    elif 'образ' in x:
        return 'образование'
    elif 'свадь' in x:
        return 'свадьба'
    elif 'жиль' in x:
        return 'жилье'
    else:
        return 0

Let's add a new column called `purpose_category`, where we will reflect the data returned by the `df_purpose_fc` function.

In [312]:
df['purpose_category'] = df['purpose'].apply(df_purpose_fc)
df_purpose = df.groupby(by='purpose_category')['debt'].sum() / df.groupby(by='purpose_category')['debt'].count() * 100
df_purpose

purpose_category
авто            9.359034
жилье           6.905830
недвижимость    7.463392
образование     9.220035
свадьба         8.003442
Name: debt, dtype: float64

**Conclusion**

Clients are more likely to have overdue loans if their loan purpose is related to any car-related operation.

On the other hand, clients whose loan purpose is to purchase housing tend to be more conscientious about repaying the loan compared to clients with other purposes.


## Step 4. General Conclusion

Analyzing credit data of over 20,000 individuals, the following conclusions can be drawn:

1. Having at least one child in the family reduces the chances of repaying the loan on time.
2. Marital status also affects loan repayment. Singles or those in civil marriages are at risk. If a loan is to be issued, it is better to choose from the categories of "widows/widowers" or "divorced".
3. Income level affects loan repayment. The ideal scenario for a loan is an individual with a total income ranging from 500k to 1,000k. The risk group includes individuals earning from 100k to 500k. Individuals in this category are significantly more likely to fail to repay the loan on time.
4. The purpose of the loan also affects repayment. The best purpose is to purchase housing. The purpose with a higher risk of overdue payments is related to car purchases.